In [46]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
!pip install holidays
import holidays
import statsmodels.api as sm

     |████████████████████████████████| 121 kB 7.3 MB/s eta 0:00:01
     |████████████████████████████████| 45 kB 6.9 MB/s  eta 0:00:01
     |████████████████████████████████| 732 kB 9.7 MB/s eta 0:00:01
  Created wheel for holidays: filename=holidays-0.10.5.2-py3-none-any.whl size=126812 sha256=46dc4166635ebb782ef88d0c57d1691fe0971ca1ed74021bc0bf287e4fca9371
  Stored in directory: /Users/bhavya/Library/Caches/pip/wheels/6a/cf/7a/3ffc6ba5930fd024335b4a12b0be8bc3ba5a8fa63ca9a849ef
  Created wheel for pymeeus: filename=PyMeeus-0.3.7-py3-none-any.whl size=702876 sha256=320a464279a8326d12031055292f7f9f05dc7c6c18227ab68e6a98b5177a39d6
  Stored in directory: /Users/bhavya/Library/Caches/pip/wheels/5a/68/50/d989a005ecd4f58a7922bede25ff7e391d66395a3090acf97a
Successfully built holidays pymeeus


#### Least Squares Calibration


paper_params = {
    "r": 0.05,
    "a": 0.5,
    "b": 0.05,
    "theta": 0.025,
    "sigma": 0.08,
    "lambda": -0.5,
    "gamma": 0.01,
    "h": 10,
    "mu": 0
}

https://www.statisticshowto.com/wp-content/uploads/2016/01/Calibrating-the-Ornstein.pdf
    
https://victor-bernal.weebly.com/uploads/5/3/6/9/53696137/projectcalibration.pdf

In [174]:
pd.set_option('precision', 9)

In [212]:
rates = pd.read_excel('data_20190228.xlsx', index_col='Date', parse_dates=True)
output = pd.read_excel('libor_rates_output_022819.xlsx', index_col='Date',parse_dates=True)
rates['Mid'] = (rates['Bid'] + rates['Ask']) / 2

In [213]:
rates.head()

,Term1,Unit,Ticker,Bid,Ask,Spread,Bid Spr Val,Ask Spr Val,Final Bid Rate,Final Ask Rate,Rate Type,Daycount,Freq,Bid+Ask,Mid
Date,,,,,,,,,,,,,,,
2019-05-31,3,MO,US0003M,2.615130000,2.615130000,NaN,0,0,2.615130000,2.615130000,Cash Rates,ACT/360,0,5.230260000,2.615130000
2019-06-19,20190619,ACTDATE,EDH19,2.599937333,2.599937333,NaN,0,0,2.599937333,2.599937333,Contiguous Futures,ACT/360,0,5.199874666,2.599937333
2019-09-18,20190918,ACTDATE,EDM9,2.604392246,2.604392246,NaN,0,0,2.604392246,2.604392246,Contiguous Futures,ACT/360,0,5.208784492,2.604392246
2019-12-18,20191218,ACTDATE,EDU9,2.608531531,2.608531531,NaN,0,0,2.608531531,2.608531531,Contiguous Futures,ACT/360,0,5.217063062,2.608531531
2020-03-18,20200318,ACTDATE,EDZ9,2.637362211,2.637362211,NaN,0,0,2.637362211,2.637362211,Contiguous Futures,ACT/360,0,5.274724421,2.637362211


In [214]:
output.head()

,Zero Rate,Forward Rate
Date,,
2019-03-04,0.000000000,2.615130000
2019-06-04,2.682176792,2.592484971
2019-09-04,2.670525315,2.607773819
2019-12-04,2.662163079,2.632145692
2020-03-04,2.664183657,2.578889078


In [177]:
def compute_dt(t1, t2, day_count='ACT/360'):
    if day_count == 'ACT/360':
        return (t2 - t1).days / 360
    elif day_count == 'ACT/365':
        return (t2 - t1).days / 365
    else:
        D1 = 30 if t1.day == 31 else t1.day
        D2 = 30 if t2.day == 31 and (D1 == 30 or D1 == 31) else t2.day
        return (360*(t2.year-t1.year) + 30*(t2.month-t1.month) + (D2-D1)) / 360
def compute_DF_from_rs(dt):
    r_s.loc[dt, 'DF'] = 1 / (1 + r_s.loc[dt, 'rs']*r_s.loc[dt, 't1'])
def compute_rs_from_DF(dt):
    r_s.loc[dt, 'rs'] = (1 / r_s.loc[dt, 'DF'] - 1) / r_s.loc[dt, 't1']

In [178]:
idx = pd.date_range(start=output.index.min(), end=output.index.max())
r_s = pd.DataFrame(index=idx, columns=['rs', 't1', 'DF'])
r_s['t1'] = (r_s.index - r_s.index[0]).days / 360 # ACT/360
spot_date = r_s.iloc[0].name

In [179]:
 # 3-month libor
l_end = rates.iloc[0].name # libor end date
r_s.loc[l_end, 'rs'] = rates.iloc[0]['Mid'] / 100
# extrapolate short end
r_s.loc[:l_end, 'rs'] = r_s.loc[l_end, 'rs']
r_s.loc[:l_end, 'DF'] = 1 / (1 + r_s.loc[:l_end, 'rs']*r_s.loc[:l_end, 't1'])

In [154]:
rates.iloc[1].name

'2019-06-19 00:00:00'

In [180]:
# Eurodollar futures
rt = pd.to_datetime('20190320') # roll date
#x = pd.to_datetime(rates.iloc[1].name)- rt
#print(x.days)
for i in range(1, 7):
    #print(i)
    rsf = rates.iloc[i]['Mid'] / 100 # futures rate
# rcf = rates.iloc[i]['Mid'] / 100 # futures rate
    #print(rt, pd.to_datetime(rates.iloc[i].name))#, (pd.to_datetime(rates.iloc[i].name)-rt))
    dt = compute_dt(pd.to_datetime(rt), pd.to_datetime(rates.iloc[i].name))
    r_s.loc[rates.iloc[i].name, 'DF'] = r_s.loc[rt, 'DF'] / (1 + rsf*dt)
    compute_rs_from_DF(rates.iloc[i].name)
    rt = rates.iloc[i].name
f_end = rates.iloc[6].name # end of futures

In [181]:
r_s.loc[l_end:f_end, 'rs'] = r_s.loc[l_end:f_end, 'rs'].astype('float').interpolate(method='time')
r_s.loc[l_end:f_end, 'DF'] = 1 / (1 + r_s.loc[l_end:f_end, 'rs']*r_s.loc[l_end:f_end, 't1'])

In [182]:
def DF_func(x, end, df, rsw):
    df_ = df.copy()
    df_.loc[end, 'DF'] = x # guess DF
    df_.loc[end, 'rs'] = (1/x - 1) / df_.loc[end, 't1'] # guess r_s
    df_['rs'] = df_['rs'].astype('float64').interpolate(method='time')
    df_['DF'] = 1 / (1 + df_['rs'] * r_s['t1'])
    return (df_['DF']*df_['tp']).sum()*rsw + x - 1

In [183]:
# Swap
from pandas.tseries.holiday import USFederalHolidayCalendar
period = 6 # semi-annual
bday_us = pd.offsets.CustomBusinessDay(n=0, calendar=USFederalHolidayCalendar())
s_end = rates.iloc[-1].name
months = (pd.to_datetime(s_end).year - pd.to_datetime(spot_date).year)*12 + pd.to_datetime(s_end).month - pd.to_datetime(spot_date).month
pmt_idx = pd.DatetimeIndex([spot_date + pd.DateOffset(months=j) for j in range(0, months+period, period)])
pmt_dates = (pmt_idx + bday_us).to_series().shift(1)[1:]
pmt_period = (360*(pmt_dates.index.year-pmt_dates.dt.year) +30*(pmt_dates.index.month-pmt_dates.dt.month) +(pmt_dates.index.day-pmt_dates.dt.day)) / 360
r_s['tp'] = pmt_period

In [184]:
from scipy.stats import norm
from scipy.optimize import bisect, curve_fit, minimize

In [185]:
rates.head(20)

,Term1,Unit,Ticker,Bid,Ask,Spread,Bid Spr Val,Ask Spr Val,Final Bid Rate,Final Ask Rate,Rate Type,Daycount,Freq,Bid+Ask,Mid
Date,,,,,,,,,,,,,,,
2019-05-31,3,MO,US0003M,2.615130000,2.615130000,NaN,0,0,2.615130000,2.615130000,Cash Rates,ACT/360,0,5.230260000,2.615130000
2019-06-19,20190619,ACTDATE,EDH19,2.599937333,2.599937333,NaN,0,0,2.599937333,2.599937333,Contiguous Futures,ACT/360,0,5.199874666,2.599937333
2019-09-18,20190918,ACTDATE,EDM9,2.604392246,2.604392246,NaN,0,0,2.604392246,2.604392246,Contiguous Futures,ACT/360,0,5.208784492,2.604392246
2019-12-18,20191218,ACTDATE,EDU9,2.608531531,2.608531531,NaN,0,0,2.608531531,2.608531531,Contiguous Futures,ACT/360,0,5.217063062,2.608531531
2020-03-18,20200318,ACTDATE,EDZ9,2.637362211,2.637362211,NaN,0,0,2.637362211,2.637362211,Contiguous Futures,ACT/360,0,5.274724421,2.637362211
2020-06-17,20200617,ACTDATE,EDH0,2.565892358,2.565892358,NaN,0,0,2.565892358,2.565892358,Contiguous Futures,ACT/360,0,5.131784716,2.565892358
2020-09-16,20200916,ACTDATE,EDM0,2.514128416,2.514128416,NaN,0,0,2.514128416,2.514128416,Contiguous Futures,ACT/360,0,5.028256831,2.514128416
2021-02-28,2,YR,USSWAP2,2.609233379,2.612165928,NaN,0,0,2.609233379,2.612165928,Swap Rates,30I/360,2,5.221399307,2.610699654
2022-02-28,3,YR,USSWAP3,2.566385269,2.569414139,NaN,0,0,2.566385269,2.569414139,Swap Rates,30I/360,2,5.135799408,2.567899704


In [186]:
prev_end = pd.to_datetime(f_end)
print(prev_end)
for j in range(7, 24):
    print(j, rates.iloc[j].name)
    end = pd.to_datetime(rates.iloc[j].name)
    print(pd.to_datetime(rates.iloc[j].name))
    pt_idx = pmt_dates[pd.to_datetime(pd.to_datetime(spot_date)):pd.to_datetime(end)].index
    df = r_s.loc[pt_idx]
    rsw = rates.iloc[j]['Mid'] / 100 # swap rate
    #print(df)
# binary search to find DF
    r_s.loc[end, 'DF'] = bisect(DF_func, 1e-12, 1, args=(pd.to_datetime(end), df, rsw),xtol=1e-12)
    r_s.loc[end, 'rs'] = (1/r_s.loc[pd.to_datetime(end), 'DF'] - 1) / r_s.loc[end, 't1']
# interpolate along the way
    r_s.loc[prev_end:end, 'rs'] = r_s.loc[prev_end:end, 'rs'].astype('float').interpolate(method='time')
    r_s.loc[prev_end:end, 'DF'] = 1 / (1 + r_s.loc[prev_end:end, 'rs']*r_s.loc[prev_end:end, 't1'])
    prev_end = end

2020-09-16 00:00:00
7 2021-02-28 00:00:00
2021-02-28 00:00:00
8 2022-02-28 00:00:00
2022-02-28 00:00:00
9 2023-02-28 00:00:00
2023-02-28 00:00:00
10 2024-02-29 00:00:00
2024-02-29 00:00:00
11 2025-02-28 00:00:00
2025-02-28 00:00:00
12 2026-02-28 00:00:00
2026-02-28 00:00:00
13 2027-02-28 00:00:00
2027-02-28 00:00:00
14 2028-02-29 00:00:00
2028-02-29 00:00:00
15 2029-02-28 00:00:00
2029-02-28 00:00:00
16 2030-02-28 00:00:00
2030-02-28 00:00:00
17 2031-02-28 00:00:00
2031-02-28 00:00:00
18 2034-02-28 00:00:00
2034-02-28 00:00:00
19 2039-02-28 00:00:00
2039-02-28 00:00:00
20 2044-02-29 00:00:00
2044-02-29 00:00:00
21 2049-02-28 00:00:00
2049-02-28 00:00:00
22 2059-02-28 00:00:00
2059-02-28 00:00:00
23 2069-02-28 00:00:00
2069-02-28 00:00:00


In [187]:
# extrapolate long end
r_s.loc[prev_end:, 'rs'] = r_s.loc[prev_end, 'rs']
r_s.loc[prev_end:, 'DF'] = 1 / (1 + r_s.loc[prev_end:, 'rs']*r_s.loc[prev_end:,'t1'])

In [188]:
r_s['t2'] = (360*(r_s.index.year-r_s.index[0].year) +30*(r_s.index.month-r_s.index[0].month) +(r_s.index.day-r_s.index[0].day)) / 360
r_s['zero'] = ((1/r_s['DF'])**(1/(r_s['t2']*2)) - 1)*200

In [189]:
result = output.merge(r_s[['DF', 'zero', 't1']], how='left', left_index=True,right_index=True)

In [190]:
result['date'] = result.index
result['t3'] = (result['date'].shift(-1) - result['date']).dt.days
result['forward'] = 100 * (1 / (result['DF'].shift(-1) / result['DF']) - 1) /(result['t3']/360)

In [192]:
result['diff_zero'] = 1e9 * abs(result['zero'] - result['Zero Rate'])
result['diff_forward'] = 1e9 * abs(result['forward'] - result['Forward Rate'])
pd.set_option('display.max_rows', result.shape[0])
df_final=result.loc[:'20690904', ['DF', 'zero', 'Zero Rate', 'forward', 'Forward Rate','diff_zero', 'diff_forward']]

In [193]:
df_final.head()

,DF,zero,Zero Rate,forward,Forward Rate,diff_zero,diff_forward
Date,,,,,,,
2019-03-04,1,0,0.000000000,2.61295087,2.615130000,0,2179132.06
2019-06-04,0.993366753,2.67993435,2.682176792,2.59466296,2.592484971,2242438.02,2177990.31
2019-09-04,0.986823319,2.67052479,2.670525315,2.6077725,2.607773819,529.205833,1316.99636
2019-12-04,0.980360908,2.66216228,2.662163079,2.63214608,2.632145692,799.628717,389.975789
2020-03-04,0.973881209,2.66418316,2.664183657,2.57888945,2.578889078,500.497424,376.193776


In [209]:
X = np.array(df_final.iloc[1:,2].shift(1))[1:]
y = np.array(df_final.iloc[1:,2].iloc[1:])


In [210]:
X = sm.add_constant(X)
model = sm.OLS(y, X)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.998
Model:                            OLS   Adj. R-squared:                  0.998
Method:                 Least Squares   F-statistic:                 1.016e+05
Date:                Wed, 10 Feb 2021   Prob (F-statistic):          1.72e-271
Time:                        17:21:16   Log-Likelihood:                 795.42
No. Observations:                 201   AIC:                            -1587.
Df Residuals:                     199   BIC:                            -1580.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0141      0.009      1.597      0.1

In [211]:
dt = 1/365
lam = (1-results.params[1])/dt
mu = results.params[0]/(1-results.params[1])
sig = np.std(results.resid)*np.sqrt(-2*np.log(results.params[1])/((dt)*(1-(results.params[1]**2))))
print(lam,mu,sig)

1.7612552420229377 2.916425463877084 0.08857670593183221
